In [ ]:
from pykafka import KafkaClient
import time

In [ ]:
client = KafkaClient(hosts="127.0.0.1:9092")

In [ ]:
topic = client.topics['geostream']
producer = topic.get_sync_producer()

In [ ]:
with topic.get_sync_producer() as producer:
    i = 0
    while True:
        i = i + 1
        producer.produce(('test message ' + str(i ** 2)).encode("ascii"))
        time.sleep(1)
        